# PQMF - filterbank decomposition

In [ ]:
# Import necessary libraries
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12,3)

In [ ]:
def mag_DFT(x, points=None):
    L = (points or len(x)) // 2
    points = 2 * L + 1
    w = 2 * np.pi * np.arange(-L, L+1) / points
    return w, np.abs(np.fft.fftshift(np.fft.fft(x, points)))

The following functions creates the PQMF modulated filterbank; you can skip the details

In [ ]:
def PQMF(K, L):
    # K: number of bands
    # length of impulse response will be M=2L+1
    
    M = 2 * L + 1
    w = np.kaiser(M, 10)
    n = np.arange(-L, L + 1)

    fft_length = int(2 ** np.ceil(np.log(M) / np.log(2)))
    index = fft_length // (4 * K)
    omega = np.pi / (2 * K)
        
    step_size = 1e-2
    decay = 0.5
    min_error = np.inf
    for _ in range(100):
        with np.errstate(invalid="ignore"):
            h = np.sin(omega * n) / (np.pi * n)
        h[L] = omega / np.pi
        prototype_filter = h * w
        H = np.fft.rfft(prototype_filter, n=fft_length)
        error = np.square(np.abs(H[index])) - 0.5
        
        abs_error = np.abs(error)
        if abs_error < 1e-6:
            break  # convergence
        if abs_error < min_error:
            min_error = abs_error
        else:
            step_size *= decay
            
        omega -= np.sign(error) * step_size

    filters = []
    for k in range(K):
        a = ((2 * k + 1) * np.pi / (2 * K)) * n
        b = (-1) ** k * (np.pi / 4)
        c = 2 * prototype_filter
        filters.append(c * np.cos(a + b))

    return np.asarray(filters)


## Four-band filterbank

In [ ]:
BANDS = 4

fb = PQMF(BANDS, 127)
for h in fb:
    plt.plot(*mag_DFT(h, 1024))

## Create a test signal with one spectral line per band

In [ ]:
N = 1000
x = np.zeros(N)
for k in range(BANDS):
    x += np.cos(np.pi / BANDS * (k + 0.5) * np.arange(N) + k * np.pi) / (k+1) 

s = slice(200,300)
plt.plot(x[s], 'k');

In [ ]:
for h in fb:
    plt.plot(*mag_DFT(h, 1024))
plt.plot(*mag_DFT(x / N * 2), 'k');

## Subband decomposition

In [ ]:
sbs = np.zeros((BANDS, len(x)))
for k in range(BANDS):
    sbs[k] = sp.lfilter(fb[k], [1], x)

## Subband signals before downsampling

Before downsamping, each subband signal contains a single spectral line at the original frequency

In [ ]:
for k in range(BANDS):
    plt.subplot(2,2,k+1)
    plt.plot(sbs[k][s], f'C{k}')
plt.tight_layout()

In [ ]:
for k in range(4):
    plt.subplot(2,2,k+1)
    plt.plot(*mag_DFT(sbs[k]), f'C{k}')
plt.tight_layout()

## Subband signals after downsampling

After downsampling, all subband content is automatically demodulated, so the four oscillation (which oiginally were in the middle of their respective subbands) appear to be at the same frequency as the oscillation in the first subband 

In [ ]:
for k in range(BANDS):
    plt.subplot(2,2,k+1)
    plt.plot(sbs[k][s][::BANDS], f'C{k}')
plt.tight_layout()